# Optimizing A Simple DNN Training Program Using Nsight Systems

## 4.1 Profile the application
For the first step in our optimization workflow, we will use the command line interface (CLI) of Nsight Systems to profile our application. Since it takes ~2minutes to execute the 10 epochs in the application, let's limit the profiling to the second epoch. The first epoch is meant to warm up the GPU and to get things going so that the second epoch is representative of the rest of the epochs.

We can do this by using the APIs `cudaProfilerStart` and `cudaProfilerStop` in the program to mark the region of interest as shown below. These APIs are a no-op when there is no profiler present, but they act as programmatic markers for the profiler to start/stop the collection.

<img src=images/cudaProfilerApi.jpg width=50%>

Please execute the following command to profile the second epoch of the application.

In [ ]:
!nsys profile --trace cuda,osrt \
--capture-range cudaProfilerApi \
--output /dli/task/nsys/baseline \
--force-overwrite true \
python3 /dli/task/nsys/application/main_baseline.py

While the collection is running, let's look at the command line we used.

<img src=images/Commandline.jpg width=60%>

Once the profiling is done we will inspect the report in the Nsight Systems' GUI. Execute the following cell to generate the URL for a remote desktop where you can run the Nsight Systems' GUI. Copy the URL and paste into a new browser tab. 

In [ ]:
%%js
var url = window.location.hostname + '/nsight/';
element.append(url)

Click on ... to view the notes from the instructor's presentation.

1. Copy the URL and paste into a new browser tab.
2. When prompted, enter the password as `nvidia`.
3. In the remote desktop, double click on the NVIDIA Nsight Systems desktop icon.
4. When prompted for the permission to collect telemetry, use the left/right arrow keys to select your answer and use the enter key to make your selection. Clicking 'yes' will report telemetry and crashes to the development team and help improve the product.
5. When prompted about the Options Preset, select the "CPU Rows on Top (classic)" mode.
5. Either use the Ctrl+O keyboard shortcut or click “File-> Open" menu option to open the /root/Desktop/reports/nsys/baseline.qdrep file.
6. Make sure to leave this browser tab open since you will be going back to it frequently to inspect more reports as we progress through the lab.

## 4.2 Analyze the Profile

The timeline of the application is as shown below.

<img src=images/Baseline.jpg>

Hovering over the blue chart in the CUDA device row, we see that the CUDA kernel coverage on the GPU is about 30-40% throughout. This means that the GPU is idle for the remaining 60-70% of the time.

<img src=images/BaselineCudaKernelCoverage.jpg>

**Let's zoom in to the timeline to see what's going on.** Press the Ctrl key while moving the mouse scroll wheel up or down to zoom into or out of the area around the mouse pointer. Another way to zoom in is to select the region you want to zoom into and press Shift+Z keys.

<img src=images/BaselineZoomedIn.jpg>

Clearly there is a repeating pattern where the GPU is idle for some time followed by a burst of kernel and memory operations. During the GPU idle period, some of the CPU threads are performing read/write/mmap64 operations or waiting on a condition variable. This is a classic example of GPU starvation due to sluggish CPU threads.

To identify which step in the CPU algorithm is slowing the GPU down, let's add annotations to the program to mark the phases of the training algorithm. To do so, we will use [NVIDIA Tools Extension](https://nvidia.github.io/NVTX/doxygen/index.html) (NVTX) library which provides a set of functions for annotating events, code ranges, and resources in your application. The NVTX APIs provide additional information to NVIDIA’s tools while incurring almost no overhead when the tool is not attached to the application. Sample annotations look like the following.

<img src=images/NVTX_annotations.jpg width=50%>

To view the full list of changes to add NVTX annotations, execute the cell below to compare the [main_baseline.py](application/main_baseline.py) and [main_baseline_nvtx.py](application/main_baseline_nvtx.py) files.

In [ ]:
!diff -y -d --color=always application/main_baseline.py application/main_baseline_nvtx.py

Let's profile again to get the newly added NVTX annotations by executing the cell given below.

In [ ]:
!nsys profile --trace cuda,osrt,nvtx \
--capture-range cudaProfilerApi \
--output /dli/task/nsys/baseline_nvtx \
--force-overwrite true \
python3 /dli/task/nsys/application/main_baseline_nvtx.py

Open the report with NVTX annotations (baseline_nvtx.qdrep file) in the GUI and zoom in to find the repeating pattern of 'GPU idle' followed by 'a burst of kernel and memory operations'. From the NVTX annotation, we can see that that the GPU is idle during the "Data loading" phase of the training algorithm.

<img src=images/StarvationDuringDataLoading.jpg>

## 4.3 Optimize Code to Address the CPU Bottleneck
From analyzing the profile with NVTX annotations, it is clear that the data loading step of the training algorithm is slowing the GPU down. Let's inspect the data loader [torch.utils.data.DataLoader](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) used in our program. From the code in [main_baseline_nvtx.py](application/main_baseline_nvtx.py):

<img src=images/NumberOfWorkers.jpg width=50%>

A single worker subprocess is asynchronously loading the data. In order to increase the overlap between data loading and training on the GPU, the `num_workers` parameter should be increased. Execute the following cell to see the code changes made to tune this parameter

In [ ]:
!diff -d --color=always application/main_baseline_nvtx.py application/main_opt1.py

Depending on the number of CPU cores available on the target system, we can increase `num_of_workers` to improve the overlap.

## 4.4 Profile to Verify Optimization
Let's profile again by executing the below cell to verify if the code change addresses the bottleneck.

In [ ]:
!nsys profile --trace cuda,osrt,nvtx \
--capture-range cudaProfilerApi \
--output /dli/task/nsys/firstOptimization \
--force-overwrite true \
python3 /dli/task/nsys/application/main_opt1.py

Open the report (firstOptimization.qdrep) in the GUI and zoom into the data loading phase of one of the batches. Comparing the profiles from before and after the optimization, we notice that the data loading phase's duration reduced to ~1ms from ~5ms. This might not seem like a lot, but it adds up when you consider hundreds of batches over each epoch in the training algorithm.

<img src=images/Optimization1.jpg>

## 4.5 Recap
In this section you learnt how to do the following
- Use the command line interface of Nsight Systems to profile an application
- Use cudaProfilerStart/Stop APIs to limit the profiling to just a section of the application
- Use NVTX APIs in the application to add more context about the application's algorithms to the timeline data presented by Nsight Systems
- A strategy to identify the root cause of time gaps where the GPU is idle.

Now that we have addressed one bottleneck, let's identify the next one in [step 05](05_cuda_memcpy_async.ipynb)